In [0]:
# Query 1. - Add a column named as "Revenue" into dataframe which is the sum of the below columns 'Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount'
# Query 2. - Increasing count of total passengers in New York City by area
# Query 3. - Realtime Average fare/total earning amount earned by 2 vendors
# Query 4. - Moving Count of payments made by each payment mode
# Query 5. - Highest two gaining vendor's on a particular date with no of passenger and total distance by cab
# Query 6. - Most no of passenger between a route of two location.
# Query 7. - Get top pickup locations with most passengers in last 5/10 seconds.

In [0]:
data_path="/FileStore/tables/yellow_tripdata_2018_01.parquet"

In [0]:
df=spark.read.parquet(data_path)
display(df.limit(5))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null


In [0]:
from pyspark.sql.functions import *
df=df.withColumn("Revenue",col('Fare_amount')+col('Extra')+col('MTA_tax')+col('Improvement_surcharge')+col('Tip_amount')+col('Tolls_amount')+col('Total_amount'))
display(q1_df.limit(5))

#df=df.withColumn("Revenue", agg(col(sum('Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount')) )

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Revenue
1,2018-01-01T00:21:05.000+0000,2018-01-01T00:24:23.000+0000,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null,null,11.6
1,2018-01-01T00:44:55.000+0000,2018-01-01T01:03:05.000+0000,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.0,0.0,0.3,15.3,null,null,30.6
1,2018-01-01T00:08:26.000+0000,2018-01-01T00:14:21.000+0000,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null,16.6
1,2018-01-01T00:20:22.000+0000,2018-01-01T00:52:51.000+0000,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.0,0.0,0.3,34.8,null,null,69.6
1,2018-01-01T00:09:18.000+0000,2018-01-01T00:27:06.000+0000,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55,null,null,33.1


In [0]:
q2_df=(df.groupby("PULocationID", "DOLocationID").sum("passenger_count"))
display(q2_df.limit(5))

PULocationID,DOLocationID,sum(passenger_count)
79,116,580
246,249,5073
234,144,7047
161,193,100
231,261,5812


In [0]:
# Query 3. - Realtime Average fare/total earning amount earned by 2 vendors
q3_df=(df.groupby("VendorID")
       .agg(sum('fare_amount'), avg('total_amount')))
    
display(q3_df)

VendorID,sum(fare_amount),avg(total_amount)
1,4.599966776999971E7,15.127384289902137
2,6.126991590999834E7,15.775723474073514


In [0]:
q4_df=df.groupby("payment_type").agg(count("passenger_count"))
display(q4_df.limit(5))

payment_type,count(passenger_count)
1,6106416
3,43204
2,2599215
4,11852


In [0]:
# Query 5. - Highest two gaining vendor's on a particular date with no of passenger and total distance by cab
# q5df=df.groupby(to_date("tpep_pickup_datetime"))
# .count("passenger_count")
# .agg(sum("trip_distance"))

df = df.withColumn("date",to_date("tpep_pickup_datetime"))
q5_df = (df.groupBy("VendorID","date")
         .agg(sum("passenger_count").alias("total_passenger"),
              sum("trip_distance").alias("total_distance"),
              sum("Revenue").alias("total_revenue")))
display(q5_df.sort(desc("total_revenue")).limit(2))

VendorID,date,total_passenger,total_distance,total_revenue
2,2018-01-25,351475,518394.0400000009,5966751.749995339
2,2018-01-26,354497,513454.3299999982,5927691.89999487


In [0]:
# Query 6. - Most no of passenger between a route of two location.
q6_df=(df.groupby("PULocationID","DOLocationID")
     .agg(sum("passenger_count")))
display(q6_df.limit(5))

PULocationID,DOLocationID,sum(passenger_count)
79,116,580
246,249,5073
234,144,7047
161,193,100
231,261,5812


In [0]:
#Query 7. - Get top pickup locations with most passengers in last 5/10 seconds.
from datetime import timedelta
latest_entry_time=df.agg(max(col('tpep_pickup_datetime'))).collect()
required_time=latest_entry_time[0][0]-timedelta(seconds=5)
q7_df=(df.filter(col("tpep_pickup_datetime")>=required_time)
      .groupby("PULocationID")
      .sum("passenger_count"))

In [0]:
display(q7_df)

PULocationID,sum(passenger_count)
48,2


In [0]:
# #Query 7. - Get top pickup locations with most passengers in last 5/10 seconds.
# q7_df=df.withColumn("time", date_format('tpep_pickup_datetime', 'HH:mm:ss')).withColumn("date", to_date('tpep_pickup_datetime'))
# q7_df=q7_df.filter(col("Date")=="2018-07-27").sort(desc(col("time")))
# display(q7_df)
# # date_format('datetime', 'HH:mm:ss')
# # q7_df=q7_df.sort(col("date").desc())

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,time,date
2,2018-07-27T04:06:37.000+0000,2018-07-27T04:46:57.000+0000,2,17.72,2,N,48,132,1,52.0,0.0,0.5,10.56,0.0,0.3,63.36,null,null,04:06:37,2018-07-27
2,2018-07-27T03:55:27.000+0000,2018-07-27T03:58:08.000+0000,1,0.72,1,N,48,246,2,4.0,0.0,0.5,0.0,0.0,0.3,4.8,null,null,03:55:27,2018-07-27
2,2018-07-27T03:34:24.000+0000,2018-07-27T03:46:39.000+0000,1,3.36,1,N,231,100,2,12.5,0.0,0.5,0.0,0.0,0.3,13.3,null,null,03:34:24,2018-07-27
2,2018-07-27T03:16:59.000+0000,2018-07-27T03:23:37.000+0000,1,1.75,1,N,234,125,2,7.5,0.0,0.5,0.0,0.0,0.3,8.3,null,null,03:16:59,2018-07-27
2,2018-07-27T02:42:00.000+0000,2018-07-27T02:46:56.000+0000,1,1.16,1,N,246,68,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null,02:42:00,2018-07-27
2,2018-07-27T02:22:13.000+0000,2018-07-27T02:28:28.000+0000,1,1.21,1,N,143,50,2,6.5,0.5,0.5,0.0,0.0,0.3,7.8,null,null,02:22:13,2018-07-27


In [0]:
#display(q7_df.sort(col("date").desc()))

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,time,date
2,2018-07-27T03:34:24.000+0000,2018-07-27T03:46:39.000+0000,1,3.36,1,N,231,100,2,12.5,0.0,0.5,0.0,0.0,0.3,13.3,null,null,03:34:24,2018-07-27
2,2018-07-27T02:42:00.000+0000,2018-07-27T02:46:56.000+0000,1,1.16,1,N,246,68,1,6.0,0.5,0.5,1.0,0.0,0.3,8.3,null,null,02:42:00,2018-07-27
2,2018-07-27T03:55:27.000+0000,2018-07-27T03:58:08.000+0000,1,0.72,1,N,48,246,2,4.0,0.0,0.5,0.0,0.0,0.3,4.8,null,null,03:55:27,2018-07-27
2,2018-07-27T04:06:37.000+0000,2018-07-27T04:46:57.000+0000,2,17.72,2,N,48,132,1,52.0,0.0,0.5,10.56,0.0,0.3,63.36,null,null,04:06:37,2018-07-27
2,2018-07-27T02:22:13.000+0000,2018-07-27T02:28:28.000+0000,1,1.21,1,N,143,50,2,6.5,0.5,0.5,0.0,0.0,0.3,7.8,null,null,02:22:13,2018-07-27
2,2018-07-27T03:16:59.000+0000,2018-07-27T03:23:37.000+0000,1,1.75,1,N,234,125,2,7.5,0.0,0.5,0.0,0.0,0.3,8.3,null,null,03:16:59,2018-07-27
2,2018-02-01T00:02:48.000+0000,2018-02-01T00:08:48.000+0000,3,1.74,1,N,114,164,2,7.0,0.5,0.5,0.0,0.0,0.3,8.3,null,null,00:02:48,2018-02-01
2,2018-02-01T19:13:20.000+0000,2018-02-01T19:19:26.000+0000,1,1.12,1,N,237,236,1,6.0,0.5,0.5,1.82,0.0,0.3,9.12,null,null,19:13:20,2018-02-01
2,2018-02-01T00:03:13.000+0000,2018-02-01T00:12:48.000+0000,1,1.3,1,N,79,249,2,8.0,0.5,0.5,0.0,0.0,0.3,9.3,null,null,00:03:13,2018-02-01
2,2018-02-01T00:01:40.000+0000,2018-02-01T00:02:19.000+0000,1,0.09,1,N,163,230,2,2.5,0.5,0.5,0.0,0.0,0.3,3.8,null,null,00:01:40,2018-02-01


In [0]:
#display(q7_df.select('date').distinct())

date
2009-01-01
2018-01-23
2001-01-05
2018-01-11
2018-01-08
2018-01-18
2018-01-14
2018-01-05
2018-01-13
2018-02-01
